### In this script we will extract metrics for each commit. For that you need to download and install Understand Tool, available here https://scitools.com/trial-download-3/. You need to install in 'SANER2022-CodeSamples/2-ExtractingMetrics/understand'. To run you will need license to Understand, student get free in https://scitools.com/non-commercial-license/.

In [ ]:
pip install GitPython

In [ ]:
pip install JPype1

In [ ]:
def printStatus(index, size):
    print("{0}% Completed samples".format((index / size) * 100))

In [ ]:
from git import Repo
import pandas as pd
import numpy as np
import os
import fnmatch
from statistics import mean
import shutil

In [ ]:
def getCommitsFrom(project):
    projectPath = "repositories/"+project
    repository = Repo(projectPath)
    try:
        repository.git.checkout("main", "-f")
    except:
        repository.git.checkout("master", "-f")
    iterCommits = repository.iter_commits()
    commits = []
    for c in iterCommits:
        commits.append(c)
    return commits

In [ ]:
def checkoutTo(project, sha):
    projectPath = "repositories/" + project
    repository = Repo(projectPath)
    repository.git.checkout(sha, "-f")

In [ ]:
def extractMetricsWithUnderstand(owner, project):
    understandPath = "understand\\SciTools\\bin\\pc-win64\\und"
    os.system('cmd /c "{0} create -languages java {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} add {1} {2}"'.format(understandPath, "repositories/"+owner+"/"+project, project))
    os.system('cmd /c "{0} settings -metrics all {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} settings -metricsOutputFile {1}.csv {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} -quiet analyze {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} metrics {1}"'.format(understandPath, project))

In [ ]:
def method_level_metrics(dataframe, owner, project):
    dataframe = dataframe[dataframe["Kind"].str.contains("Method")]
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").mean()
    return dataframe

In [ ]:
def class_level_metrics(dataframe, owner, project):
    dataframe = dataframe[dataframe["Kind"].str.contains("Class")]
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").mean()
    return dataframe

In [ ]:
def project_level_metrics(dataframe, owner, project):
    numberJavaFiles = 0 if dataframe.empty else dataframe["Kind"].value_counts()["File"]
    dataframe = dataframe[dataframe["Kind"] == "Package"]
    packages = len(dataframe)
    dataframe.drop(dataframe.columns.difference(['CountDeclClass']), 1, inplace=True)
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").sum()
    dataframe["numberJavaFiles"] = numberJavaFiles
    dataframe["packages"] = packages
    return dataframe

In [ ]:
def readMetricsFromCsv(project):
    dtype = {'Kind': str, 'Name': str, 'AvgCyclomatic': np.float64, 'AvgCyclomaticModified': np.float64,'AvgCyclomaticStrict': np.float64, 'AvgEssential': np.float64, 'AvgLine': np.float64, 'AvgLineBlank': np.float64,'AvgLineCode': np.float64, 'AvgLineComment': np.float64, 'CountClassBase': np.float64, 'CountClassCoupled': np.float64,'CountClassCoupledModified': np.float64, 'CountClassDerived': np.float64, 'CountDeclClass': np.float64,'CountDeclClassMethod': np.float64, 'CountDeclClassVariable': np.float64,'CountDeclExecutableUnit': np.float64, 'CountDeclFile': np.float64, 'CountDeclFunction': np.float64,'CountDeclInstanceMethod': np.float64, 'CountDeclInstanceVariable': np.float64,'CountDeclMethod': np.float64, 'CountDeclMethodAll': np.float64, 'CountDeclMethodDefault': np.float64,'CountDeclMethodPrivate': np.float64, 'CountDeclMethodProtected': np.float64,'CountDeclMethodPublic': np.float64, 'CountInput': np.float64,'CountLine': np.float64, 'CountLineBlank': np.float64,'CountLineCode': np.float64, 'CountLineCodeDecl': np.float64, 'CountLineCodeExe': np.float64,'CountLineComment': np.float64, 'CountOutput': np.float64, 'CountPath': np.float64, 'CountPathLog': np.float64,'CountSemicolon': np.float64, 'CountStmt': np.float64, 'CountStmtDecl': np.float64, 'CountStmtExe': np.float64,'Cyclomatic': np.float64, 'CyclomaticModified': np.float64, 'CyclomaticStrict': np.float64, 'Essential': np.float64, 'Knots': np.float64, 'MaxCyclomatic': np.float64, 'MaxCyclomaticModified': np.float64,'MaxCyclomaticStrict': np.float64, 'MaxEssential': np.float64,'MaxEssentialKnots': np.float64,'MaxInheritanceTree': np.float64, 'MaxNesting': np.float64, 'MinEssentialKnots': np.float64,'PercentLackOfCohesion': np.float64, 'PercentLackOfCohesionModified': np.float64,'RatioCommentToCode': np.float64, 'SumCyclomatic': np.float64, 'SumCyclomaticModified': np.float64,'SumCyclomaticStrict': np.float64, 'SumEssential': np.float64}
    dataframe = pd.read_csv(project+".csv", dtype=dtype)
    return dataframe

In [ ]:
def getUnderstandMetrics(owner, project):
    extractMetricsWithUnderstand(owner, project)
    
    dataframe = readMetricsFromCsv(project)
    
    method_level = method_level_metrics(dataframe.copy(), owner, project)
    method_level["level"] = "method"
    method_level.set_index("level")

    
    class_level = class_level_metrics(dataframe.copy(), owner, project)
    class_level["level"] = "class"
    class_level.set_index("level")    
    
    project_level = project_level_metrics(dataframe.copy(), owner, project)
    project_level["level"] = "project"
    project_level.set_index("level")
    
    return pd.concat([method_level, class_level, project_level], axis=0)

In [ ]:
def findPaths(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        if '.git' in root:
            continue
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [ ]:
def getMetrics(commit, owner, project):
    metrics = getUnderstandMetrics(owner, project)
    metrics["commitSha"] = commit.hexsha
    metrics["commitDate"] = commit.authored_datetime
    return metrics

In [ ]:
def deleteUnusedFiles(sample):
    os.remove(sample + ".csv")
    shutil.rmtree(sample + ".und")

In [ ]:
def createDirectoryIfNotExists(dirName):
    if not os.path.exists(dirName):
            os.makedirs(dirName)

In [ ]:
def replaceSamplePathForWindowsLike(sample):
    return sample.replace("/", "\\")

In [ ]:
def extractMetricsByCommit(sample, commit):
    owner, project = sample.split("\\")
    checkoutTo(sample, commit.hexsha)
    print("commit ======= " + commit.hexsha)
    metrics = getMetrics(commit, owner, project)
    return metrics

In [ ]:
def extractMetricsForAllCommits(commits, sample):
    allCommits = pd.DataFrame()
    for index, commit in enumerate(commits):
        metrics = extractMetricsByCommit(sample, commit)
        allCommits = allCommits.append(metrics, ignore_index=True)
        print("{0}% of commits completed from sample {1}".format((index/len(commits) * 100), sample))
        allCommits.to_csv("metrics\\"+sample+".csv", index=False)
    return allCommits

In [ ]:
exclude = []
def metricsByCommit(samples):
    for index, sample in enumerate(samples):
        if (sample in exclude):
            continue
        print(sample)
        sample = replaceSamplePathForWindowsLike(sample)
        
        printStatus(index+1, len(samples))
        
        createDirectoryIfNotExists("metrics")
        
        commits = getCommitsFrom(sample)
        commits.reverse()
        
        owner, project = sample.split("\\")
        
        createDirectoryIfNotExists("metrics\\"+owner)
        
        allCommits = extractMetricsForAllCommits(commits, sample)
        
        deleteUnusedFiles(project)
        
        allCommits.to_csv("metrics\\"+sample+".csv", index=False)

In [ ]:
samples = pd.read_csv("../1-GettingQuestions/sampleQuestions.csv")

In [ ]:
samples["full_name"] = samples["framework"] + "/" +samples["path"]

In [ ]:
metricsByCommit(samples["full_name"].unique())